# Metadynamics

Authors:   
[Lester Hedges -- @lohedges](https://github.com/lohedges)

**Jupyter cheat sheet**:
- To run the currently highlighted cell and move focus to the next cell, hold <kbd>&#x21E7; Shift</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To run the currently highlighted cell and keep focus in the same cell, hold <kbd>&#x21E7; Ctri</kbd> and press <kbd>&#x23ce; Enter</kbd>;
- To get help for a specific function, place the cursor within the function's brackets, hold <kbd>&#x21E7; Shift</kbd>, and press <kbd>&#x21E5; Tab</kbd>;
- You can find the full documentation at [biosimspace.org](https://biosimspace.org).


In this notebook you'll use BioSimspace to set up and run a simple [metadynamics](https://en.wikipedia.org/wiki/Metadynamics) simulation to calculate the free energy as a function of the $\phi$ and $\psi$ dihedral angles for an alanine dipeptide molecule in vacuum.

![alanine dipeptide](https://raw.githubusercontent.com/CCPBioSim/biosimspace-advanced-simulation/de3f65372b49879b788f46618e0bfef78b2559b9/metadynamics/assets/alanine_dipeptide.png)


First, let's import BioSimSpace:

In [ ]:
import BioSimSpace as BSS

Let's load the alanine dipeptide system from file:

In [ ]:
system = BSS.IO.readMolecules("input/ala.*")

Note that this is a solvated system, which contains the alanine dipeptide and a bunch of water molecules. Since we want to study the molecule in vacuum, we will extract the alanine dipeptide using a search:

In [ ]:
search = system.search("resname ALA").molecules()
print(search)

We found a single matching result. Lets extract the result from the search object:

In [ ]:
molecule = search[0]

Let's now examine each of the atoms in the molecule to make sure that it looks like we'd expect:

In [ ]:
for atom in molecule.getAtoms():
    print(atom)

The atoms that define the two dihedrals are:

$\phi$: C index=4, N index=6, CA index=8, C index=14

$\psi$: N index=6, CA index=8, C index=14, N index=16

Let's store the indices:

In [ ]:
phi_idx = [4, 6, 8, 14]
psi_idx = [6, 8, 14, 16]

We now need to construct two _collective variables_ that represent the dihedrals (torsions) above. These are the variables that will be sampled during our metadynamics simulation, allowing us to estimate the free energy of the system as a function of the dihedral angles.

In [ ]:
phi = BSS.Metadynamics.CollectiveVariable.Torsion(atoms=phi_idx)
psi = BSS.Metadynamics.CollectiveVariable.Torsion(atoms=psi_idx)

Note how we passed the _indices_ of the atoms involved in the torsion to the constructor. Since atoms are indexed relative to the molecule that they belong it is important that we get the _absolute_ atom index within the system. In this case we have a single molecule, so all is okay, but in general you should be careful. For example, atom index 2 in molecule 2 won't be atom index 2 in the system, i.e. you would have to shift the index by the number of atoms in molecule 1.

Thankfully we provide simple tools to compute this index for you. For example, for atom `CA` in the molecule:

In [ ]:
# Search for the atom by name and get the result.
atom = molecule.search("atomname CA")[0]

# Compute the index of the atom in the system from which it came.
system.getIndex(atom)

(In this case we get the same index as the molecule since the alanine dipeptide is the first molecule in the system.)

There are many other options that can be set for collective variables, such as setting the width of the Gaussian hill that is used to bias a variable, specifying lower and upper bounds for the value of the variable, and sampling on a pre-defined grid to help speed up simulations. Run `help(BSS.Metadynamics.CollectiveVariable.Torsion)` to see what's available.

We now need a protocol to describe our metadynamics simulation. Let's go with the defaults, other than increasing the run time to 3 nanoseconds to increase the amount of sampling.

In [ ]:
protocol = BSS.Protocol.Metadynamics(
    collective_variable=[phi, psi], runtime=3 * BSS.Units.Time.nanosecond
)

To see what's been set for us we can print the object:

In [ ]:
print(protocol)

Phew, that's a lot of stuff! To get more detail about how to configure the protocol run `help(BSS.Protocol.Metadynamics`).

Finally, we need a process to actually run our metadynamics simulation. BioSimSpace will automatically configure this for you with the `BSS.Metadynamics.run` function. Note that the function expects a `System` and a `Protcol` as arguments, so we convert our alanine dipeptide molecule to a single-molecule system by using `toSystem()`.

In [ ]:
process = BSS.Metadynamics.run(molecule.toSystem(), protocol)

All being well, we should now have a simulation process running in the background. Let's check that it's running:

In [ ]:
process.isRunning()

BioSimSpace supports metadynamics simulations using [GROMACS](http://www.gromacs.org) and [PLUMED](https://www.plumed.org). To see the PLUMED configuration file that was generated:

In [ ]:
process.getPlumedConfig()

We can interact with the process directly and monitor the values of the collective variables as a function of time. Re-run the cell with CTRL+Enter to see the values update.

(Note that the `getCollectiveVariable` function takes the indices of the collective variables that we pass to the `Protocol` constructor, i.e. 0 = $\phi$, 1 = $\psi$.)

In [ ]:
print(
    process.getTime(),
    process.getCollectiveVariable(0),
    process.getCollectiveVariable(1),
)

We can also plot the time evolution of the collective variables. Once again, use CTRL+Enter to re-run the see and see the plots update.

In [ ]:
BSS.Notebook.plot(
    x=process.getTime(time_series=True),
    y=process.getCollectiveVariable(0, time_series=True),
)
BSS.Notebook.plot(
    x=process.getTime(time_series=True),
    y=process.getCollectiveVariable(1, time_series=True),
)

It's also possible to compute the real-time free energy estimate. The returned `free_nrg` object is a tuple, containing lists for the values of $\phi$ and $\psi$, and the corresponding free energy estimate. We can use this $x$, $y$, and $z$ data to generate a contour plot:

In [ ]:
free_nrg = process.getFreeEnergy()
BSS.Notebook.plotContour(free_nrg[0], free_nrg[1], free_nrg[2])

Re-run the cell with CTRL+Enter to see how the free energy surface changes as the metadynamics simulation explores more of phase space.

If we are only intersted in the free energy as a function of a single collective variable, then it's possible to perform a projection by intergrating out the other variables. We can do this by passing the index of the collective variable of interest to the `getFreeEnergy` function, along with an appropriate temperature factor (in energy units) for integrating out the other variables. For example, to get the free energy as a function of $\phi$ only, then visualise as a simple $x$-$y$ plot:

In [ ]:
free_nrg_phi = process.getFreeEnergy(index=0, kt=2.5 * BSS.Units.Energy.kt)
BSS.Notebook.plot(x=free_nrg_phi[0], y=free_nrg_phi[1])

Having succesfully sampled the free energy landscape as a function of the $\phi$ and $\psi$ dihedral angles, we might next want to examine representative snapshots from the basins. To do this we can use the `sampleConfigurations` method of the `process` object. This takes a list of bounds for the values of the collective variables as an argument, along witht the maximum number of snaphots that we would like. The method returns a list of randomly sampled molecular configurations that lie within the bounds, along with a list containing the corresponding collective variable values.

Let's consider the basin just to the right in the middle of the free-energy contour plot. This lies roughly at $0.5\geq\phi\leq1.5$ and $-1.5\geq\psi\leq0.5$.

In [ ]:
# Create bounds on the values of the collective variables. None can be used whenever a value is unconstrained.
bounds = [
    (0.5 * BSS.Units.Angle.radian, 1.5 * BSS.Units.Angle.radian),
    (-1.5 * BSS.Units.Angle.radian, 0.5 * BSS.Units.Angle.radian),
]

# Get twenty randomly sampled configurations along with the collective variable values.
# Note that the method will return (None, None) if we haven't yet sampled the region of
# phase space defined by 'bounds'.
configs, colvars = process.sampleConfigurations(bounds, 20)

Let's examine the value of the collective variables for each sample to make sure they are in range:

In [ ]:
colvars

Let's now visualise the first configuration:

In [ ]:
# Create a view object for the system of interest.
view = BSS.Notebook.View(configs[0])

# View the entire system.
view.system()

Congratulations, you've just run your first metadynamics simulation using BioSimSpace.

# Exercise

The files [nacl.gro](input/nacl.gro) and [nacl.top](input/nacl.top) represent a sodium and chloride ion (one of each)  solvated in a box of water. Using the information above and the [documentation](https://biosimspace.org), set up and run a metadynamics simulation to estimate the free energy as a function of the distance between the two ions (which can be used to compute the dissociation barrier.)

**Hints**:

- Search the system for the ions by name (using `atomname`) and get their indices.
- You'll need to create a `BSS.Metadynamics.CollectiveVariable.Distance` object to define the collective variable. Since we don't want to waste valuable computational resources sampling large distances where the ions aren't interacting, use `BSS.Metadynamics.Bound` to create an upper bound at a distance of 0.6 nm. Set a force constant of 2000.
- It is useful to adjust the width of the Gaussian hills used to sample the free energy landscape so that you can resolve features of interest. For this simulation, choose a hill width of 0.02 nm.
- Metadynamics simulations can be run on a _grid_ to help reduce the computational cost as the simulation progresses. (From accumulating the bias potential.) Define a grid between 0 and 1 nm to use for this simulation.
- You can run _well-tempered_ metadynamics by setting a bias factor in the protocol. Here the height of the Gaussian hills are adaptively decreased over time such that the bias converges in the long-time limit. The bias factor should be chosen to be larger than the highest barrier you expect in the system (in units of $k_\mathrm{B}T$). In this system the largest barrier is expected to be around $5k_\mathrm{B}T$, so a bias factor of 5 should work well.
- On the notebook server we have restricted GROMACS to one MPI thread and two OpenMP threads per MPI thread. Depending on how many attendees there are, we might be able to use more resources per user to make your simulations go faster. To increase the number of OpenMP threads you can do something like the following:

```python
# Initialise a metadynamics process but don't start it.
process = BSS.Metadynamics.run(system, protocol, auto_start=False)
# Configure the GROMACS command-line arguments to use 4 OpenMP threads per MPI thread.
process.setArg("-ntomp", 4)
# Start the process.
process.start()
```

(Note that you can't set the number of OpenMP threads for a vacuum simulation, you can only partition space when there is a periodic box. Please don't use a very large number of threads or you will slow the server down for other users.)

If you get stuck, an answer can be found [here](answers/metadynamics_solution.ipynb).